In [1]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

import keras
import numpy as np
import tensorflow as tf
from time import time

from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, AveragePooling2D, MaxPooling2D

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
length = len(y)
X = X.reshape(length, 784)
X = X.reshape(length, 28, 28, 1)

x_train,x_test,y_train,y_test=train_test_split(X,y,train_size=60000,test_size=10000)
x_train,x_validation,y_train,y_validation=train_test_split(x_train,y_train,test_size=0.2, random_state=0)

print('# of training images:', x_train.shape[0])
print('# of testing images:', x_test.shape[0])
print('# of validation images:', x_validation.shape[0])

# of training images: 48000
# of testing images: 10000
# of validation images: 12000


In [3]:
# The LeNet architecture accepts a 32x32 pixel images as input, mnist data is 28x28 pixels. So to overcome that:
# Pad the images with 0s
x_train = np.pad(x_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_validation = np.pad(x_validation, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_test = np.pad(x_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')
    
print("Updated Image Shape: {}".format(x_train[0].shape))

Updated Image Shape: (32, 32, 1)


In [4]:
model = keras.Sequential()
model.add(Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(32,32,1)))
model.add(AveragePooling2D())
model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D())
model.add(Flatten())
model.add(Dense(units=120, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=84, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=10, activation = 'softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
#model.summary()

In [6]:
y_train = to_categorical(y_train)
y_validation = to_categorical(y_validation)

train_generator = ImageDataGenerator().flow(x_train, y_train, batch_size=128)
validation_generator = ImageDataGenerator().flow(x_validation, y_validation, batch_size=128)
steps_per_epoch = x_train.shape[0]//128
validation_steps = x_validation.shape[0]//128

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=10, 
                    validation_data=validation_generator, validation_steps=validation_steps, 
                    shuffle=True, callbacks=[tensorboard])

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
375/375 [==============================] - 12s 33ms/step - loss: 0.6820 - acc: 0.8372 - val_loss: 0.1236 - val_acc: 0.9636
Epoch 2/10
375/375 [==============================] - 12s 31ms/step - loss: 0.1507 - acc: 0.9547 - val_loss: 0.0842 - val_acc: 0.9745
Epoch 3/10
375/375 [==============================] - 12s 31ms/step - loss: 0.1055 - acc: 0.9683 - val_loss: 0.0675 - val_acc: 0.9799
Epoch 4/10
375/375 [==============================] - 13s 33ms/step - loss: 0.0798 - acc: 0.9764 - val_loss: 0.0552 - val_acc: 0.9842
Epoch 5/10
375/375 [==============================] - 12s 31ms/step - loss: 0.0666 - acc: 0.9808 - val_loss: 0.0592 - val_acc: 0.9832
Epoch 6/10
375/375 [==============================] - 12s 32ms/step - loss: 0.0571 - acc: 0.9830 - val_loss: 0.0480 - val_acc: 0.9858
Epoch 7/10
375/375 [==============================] - 12s 33ms/step - loss: 0.0483 - acc: 0.9850 - val_loss: 0.0474 - val_acc: 0.9850
Epoch 8/10
375

In [7]:
score = model.evaluate(x_test, to_categorical(y_test))
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 1s 89us/step
Test loss: 0.037904225386909095
Test accuracy: 0.99
